# 第 10 章：注意力机制

> 解决 Seq2Seq 信息瓶颈，为 Transformer 铺路

---

## 本章目标

- [ ] 理解注意力机制的直觉和数学
- [ ] 实现 Bahdanau 注意力和 Luong 注意力
- [ ] 可视化注意力权重
- [ ] 理解注意力与 Transformer 的关系

---

## 注意力机制的核心思想

不使用固定的上下文向量，而是让解码器**动态关注**输入的不同部分。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

---

## 第一部分：注意力公式

### 1.1 通用注意力公式

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

- **Q (Query)**: 查询向量（解码器当前状态）
- **K (Key)**: 键向量（编码器输出）
- **V (Value)**: 值向量（编码器输出）

In [ ]:
class BahdanauAttention(nn.Module):
    """
    Bahdanau 注意力 (加性注意力)
    
    score(s, h) = v^T tanh(W_s @ s + W_h @ h)
    """
    
    def __init__(self, hidden_size):
        super().__init__()
        self.W_s = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_h = nn.Linear(hidden_size, hidden_size, bias=False)
        self.v = nn.Linear(hidden_size, 1, bias=False)
    
    def forward(self, decoder_hidden, encoder_outputs):
        """
        decoder_hidden: (batch, hidden)
        encoder_outputs: (batch, src_len, hidden)
        """
        # 扩展 decoder_hidden
        src_len = encoder_outputs.size(1)
        decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        # 计算注意力分数
        energy = torch.tanh(self.W_s(decoder_hidden) + self.W_h(encoder_outputs))
        attention = self.v(energy).squeeze(2)  # (batch, src_len)
        
        # Softmax 得到权重
        weights = F.softmax(attention, dim=1)
        
        # 加权求和
        context = torch.bmm(weights.unsqueeze(1), encoder_outputs)
        context = context.squeeze(1)  # (batch, hidden)
        
        return context, weights


class LuongAttention(nn.Module):
    """
    Luong 注意力 (乘性注意力)
    
    score(s, h) = s^T @ W @ h
    """
    
    def __init__(self, hidden_size):
        super().__init__()
        self.W = nn.Linear(hidden_size, hidden_size, bias=False)
    
    def forward(self, decoder_hidden, encoder_outputs):
        """
        decoder_hidden: (batch, hidden)
        encoder_outputs: (batch, src_len, hidden)
        """
        # 计算分数: s^T @ W @ h
        query = self.W(decoder_hidden).unsqueeze(2)  # (batch, hidden, 1)
        attention = torch.bmm(encoder_outputs, query).squeeze(2)  # (batch, src_len)
        
        weights = F.softmax(attention, dim=1)
        context = torch.bmm(weights.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context, weights


# 测试
batch_size, src_len, hidden_size = 4, 10, 64

decoder_hidden = torch.randn(batch_size, hidden_size)
encoder_outputs = torch.randn(batch_size, src_len, hidden_size)

attn = BahdanauAttention(hidden_size)
context, weights = attn(decoder_hidden, encoder_outputs)

print(f"上下文向量: {context.shape}")
print(f"注意力权重: {weights.shape}")
print(f"权重和: {weights.sum(dim=1)}")

---

## 第二部分：可视化注意力

In [ ]:
def visualize_attention(weights, src_tokens, trg_tokens):
    """
    可视化注意力权重
    """
    fig, ax = plt.subplots(figsize=(10, 8))
    
    im = ax.imshow(weights, cmap='Blues')
    
    ax.set_xticks(range(len(src_tokens)))
    ax.set_yticks(range(len(trg_tokens)))
    ax.set_xticklabels(src_tokens, fontsize=12)
    ax.set_yticklabels(trg_tokens, fontsize=12)
    
    ax.set_xlabel('源序列', fontsize=14)
    ax.set_ylabel('目标序列', fontsize=14)
    ax.set_title('注意力权重矩阵', fontsize=16)
    
    plt.colorbar(im)
    plt.tight_layout()
    plt.show()


# 示例：翻译任务的注意力
src = ['I', 'love', 'you', '<eos>']
trg = ['我', '爱', '你', '<eos>']

# 模拟对齐的注意力权重
weights = np.array([
    [0.8, 0.1, 0.05, 0.05],  # 我 → I
    [0.1, 0.7, 0.1, 0.1],    # 爱 → love
    [0.05, 0.1, 0.8, 0.05],  # 你 → you
    [0.05, 0.05, 0.1, 0.8],  # <eos>
])

visualize_attention(weights, src, trg)

---

## 第三部分：与 Transformer 的关系

Transformer 中的 Self-Attention 是注意力机制的推广：

- **RNN + Attention**: 序列处理 + 注意力
- **Transformer**: 完全基于注意力，去掉 RNN

这将在下一个模块 `4.3_transformer/` 中详细讲解。

---

## 本章总结

1. **注意力核心**：动态加权，让模型"关注"输入的不同部分
2. **Bahdanau (加性)**：$v^T \tanh(W_s s + W_h h)$
3. **Luong (乘性)**：$s^T W h$
4. **可解释性**：注意力权重可视化

---

## 下一章预告

**第 11 章：时间序列预测**

👉 [11_time_series_forecasting.ipynb](./11_time_series_forecasting.ipynb)